In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [0]:
# Check if the mount point already exists
mount_point = "/mnt/tokyoolympic"
if any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
    dbutils.fs.unmount(mount_point)

# Define the configurations
configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": "client_id",
    "fs.azure.account.oauth2.client.secret": 'secret_key',
    "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tenant_id/oauth2/token"
}

# Mount the storage
dbutils.fs.mount(
    source="abfss://olympics2021-data@olympics2021tokyosai.dfs.core.windows.net",
    mount_point=mount_point,
    extra_configs=configs
)

/mnt/tokyoolympic has been unmounted.


True

In [0]:
%fs
ls "/mnt/tokyoolympic"

path,name,size,modificationTime
dbfs:/mnt/tokyoolympic/raw_data/,raw_data/,0,1722420423000
dbfs:/mnt/tokyoolympic/transformed_data/,transformed_data/,0,1722420436000


In [0]:
spark

In [0]:
athletes = spark.read.format("csv").option("header", "true").load("/mnt/tokyoolympic/raw_data/athletes.csv")
coaches = spark.read.format("csv").option("header", "true").load("/mnt/tokyoolympic/raw_data/coaches.csv")
entriesgender = spark.read.format("csv").option("header", "true").load("/mnt/tokyoolympic/raw_data/entriesgender.csv")
medals = spark.read.format("csv").option("header", "true").option("inferSchema","true").load("/mnt/tokyoolympic/raw_data/medals.csv")
teams = spark.read.format("csv").option("header", "true").option("inferSchema","true").load("/mnt/tokyoolympic/raw_data/teams.csv")

In [0]:
athletes.show(5)
coaches.show(5)
entriesgender.show(5)
medals.show(5)
teams.show(5)

+-----------------+-------+-------------------+
|       PersonName|Country|         Discipline|
+-----------------+-------+-------------------+
|  AALERUD Katrine| Norway|       Cycling Road|
|      ABAD Nestor|  Spain|Artistic Gymnastics|
|ABAGNALE Giovanni|  Italy|             Rowing|
|   ABALDE Alberto|  Spain|         Basketball|
|    ABALDE Tamara|  Spain|         Basketball|
+-----------------+-------+-------------------+
only showing top 5 rows

+---------------+-------------+----------+-----+
|           Name|      Country|Discipline|Event|
+---------------+-------------+----------+-----+
|ABDELMAGID Wael|        Egypt|  Football| NULL|
|      ABE Junya|        Japan|Volleyball| NULL|
|  ABE Katsuhiko|        Japan|Basketball| NULL|
|   ADAMA Cherif|C�te d'Ivoire|  Football| NULL|
|     AGEBA Yuya|        Japan|Volleyball| NULL|
+---------------+-------------+----------+-----+
only showing top 5 rows

+-------------------+------+----+-----+
|         Discipline|Female|Male|Tota

In [0]:
athletes.printSchema()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [0]:
coaches.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
entriesgender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: string (nullable = true)
 |-- Male: string (nullable = true)
 |-- Total: string (nullable = true)



In [0]:
entriesgender = entriesgender.withColumn("Female",col("Female").cast(IntegerType()))\
    .withColumn("Male",col("Male").cast(IntegerType()))\
    .withColumn("Total",col("Total").cast(IntegerType()))

In [0]:
entriesgender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [0]:
medals.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- TeamCountry: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [0]:
teams.printSchema()

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
# Find the top countries with the highest number of gold medals
top_gold_medal_countries = medals.orderBy("Gold", ascending=False).select("TeamCountry","Gold").show()

+--------------------+----+
|         TeamCountry|Gold|
+--------------------+----+
|United States of ...|  39|
|People's Republic...|  38|
|               Japan|  27|
|       Great Britain|  22|
|                 ROC|  20|
|           Australia|  17|
|         Netherlands|  10|
|              France|  10|
|             Germany|  10|
|               Italy|  10|
|              Canada|   7|
|              Brazil|   7|
|         New Zealand|   7|
|                Cuba|   7|
|             Hungary|   6|
|   Republic of Korea|   6|
|              Poland|   4|
|      Czech Republic|   4|
|               Kenya|   4|
|              Norway|   4|
+--------------------+----+
only showing top 20 rows



In [0]:
# Find the top countries with the highest number of Silver medals
top_Silver_medal_countries = medals.orderBy("Silver", ascending=False).select("TeamCountry","Silver").show()
# Find the top countries with the highest number of Bronze medals
top_Bronze_medal_countries = medals.orderBy("Bronze", ascending=False).select("TeamCountry","Bronze").show()

+--------------------+------+
|         TeamCountry|Silver|
+--------------------+------+
|United States of ...|    41|
|People's Republic...|    32|
|                 ROC|    28|
|       Great Britain|    21|
|               Japan|    14|
|         Netherlands|    12|
|              France|    12|
|             Germany|    11|
|               Italy|    10|
|               Spain|     8|
|             Hungary|     7|
|           Australia|     7|
|              Canada|     6|
|              Brazil|     6|
|         New Zealand|     6|
|              Sweden|     6|
|             Ukraine|     6|
|              Poland|     5|
|             Georgia|     5|
|            Colombia|     4|
+--------------------+------+
only showing top 20 rows

+--------------------+------+
|         TeamCountry|Bronze|
+--------------------+------+
|United States of ...|    33|
|                 ROC|    23|
|       Great Britain|    22|
|           Australia|    22|
|               Italy|    20|
|People's Repu

In [0]:
# Top 5 total count of medals
top_total_Medals = medals.orderBy("Total", ascending=False).select("TeamCountry","Total").show(5)

+--------------------+-----+
|         TeamCountry|Total|
+--------------------+-----+
|United States of ...|  113|
|People's Republic...|   88|
|                 ROC|   71|
|       Great Britain|   65|
|               Japan|   58|
+--------------------+-----+
only showing top 5 rows



In [0]:
athletes.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/tokyoolympic/transformed_data/athletes")
coaches.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/tokyoolympic/transformed_data/coaches")
entriesgender.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/tokyoolympic/transformed_data/entriesgender")
medals.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/tokyoolympic/transformed_data/medals")
teams.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/tokyoolympic/transformed_data/teams")
     